In [ ]:
from music21 import scale, pitch, note, stream, chord, instrument, midi, converter
import numpy as np
import scipy.io.wavfile as wave
from IPython.display import Audio

# Mock Project

Welcome!

This notebook has
* the assignment
* the materials you may use
* explanations on how you might get to your goal of producing an audio file and a visualization

## Assignment

* Deliverables:
  * audio file (1-3 minutes)
  * a visualization of your composition
* Deadline: Monday, 29th of March, at noon 

We ask you to compose a a piece from the given materials. You are free in the number of elements you want to be using. For example, if you don't want to work with the audio samples, or only with the audio samples, you may. The only rule is that you cannot add additional material, just transform what you are given. Otherwise, let your creativity go wild!

Differently from your semester project that you will work on in the second part of the semester, here we are not asking you to start off with a particular musical idea. Rather, you select musical elements and develop an (algorithmic?) compositional idea from it. So you might probably want to start by exploring the materials and what you can do to them with techniques we have seen in class or others. Communicate with your team members to see what cool ideas and sounds come up and make a plan on how to combine them. 

Visualizing your composition *before* you start hacking it together is a very, very good idea because it can guide you in the process. Naturally, the visualization that you sketch in the beginning does not have to be identical with the one that you hand in at the end. You are free to come up with any form of visualization you find effective in conveying how the materials you choose are organized in time. Also, come up with a description of your composition in one sentence and a title. 

Don't have your creativity constrained by what you already know. Instead, if you have a great idea but don't know how to realize it technically, do your own research and/or post your problem on Piazza so you can get quick advice. 

## Tools

To work with symbolic materials you can use python and music21.

To organize sound samples in time, you can use [Earsketch](https://earsketch.gatech.edu/landing/#/).

To manipulate raw audio, butcher and transform the sound samples, you can use [Audacity](https://www.audacityteam.org/).

Note that the music you produce in music21 can be exported in midi format and eventually [converted](https://www.zamzar.com/convert/midi-to-wav/) in .wav format, so that it becomes effectively a new sound sample that you can manipulate in Audacity and combine with the other samples in Earsketch. 

## Helpers

In [ ]:
def get_measure(pitches, dur=1, **kwargs):
    st = stream.Measure(**kwargs)
    for p in pitches:
        st.append(note.Note(p, quarterLength=dur))
    return st

def play(stream):
    """Shortcut to play a stream"""
    midi.realtime.StreamPlayer(stream).play()

## Materials: Pitch class collections

#### Nice Partials

Partials 5, 7, 9, 11, and 13, exemplified over C2:

In [ ]:
p5 = pitch.Pitch('E4', microtone=-14)
p7 = pitch.Pitch('Bb4', microtone=-31)
p9 = pitch.Pitch('D5', microtone=4)
p11 = pitch.Pitch('F#5', microtone=-49)
p13 = pitch.Pitch('A5', microtone=41)
partials = [p5, p7, p9, p11, p13]
partials_concrete = scale.ConcreteScale(pitches=partials)
intervals = partials_concrete.abstract.getIntervals()
intervals

In [ ]:
[round(i.semitones, 2) for i in intervals]

#### Nice scale

In [ ]:
nice_scale = [pitch.Pitch(p) for p in ('C', 'Db', 'E', 'F#', 'G#', 'A', 'B')]
get_measure(nice_scale).show()

#### Nice pitch collection

In [ ]:
nice_collection = [pitch.Pitch(p) for p in ('F', 'Gb', 'A#', 'B')]
chord.Chord(nice_collection).show()

## Materials: Samples
### Goat

In [ ]:
goat_sample_rate, goat_wav = wave.read('goat.wav')
goat_wav[:20]

In [ ]:
Audio(goat_wav.T, rate=goat_sample_rate)

### Broken Violin

In [ ]:
Audio('broken_violin.wav')

### Pacman

In [ ]:
Audio('pacman.wav')

### Underwater

In [ ]:
Audio('underwater.wav')

### Chatter

In [ ]:
Audio('chatter.wav')

# Materials: Timbres

### Built-in timbres: accordion, violin, clarinet, woodblock

You can use 4 different timbres that are built-in in music21: accordion, violin, clarinet, woodblock.

In [ ]:
melody = converter.parse('frere_jacques.mid').parts[0]
play(melody)

In [ ]:
instruments = [instrument.Accordion(), instrument.Violin(), instrument.Clarinet(), instrument.Woodblock()]

#Selecting an instrument for a stream
melody.insert(0, instruments[2]) #<----
play(melody)

Once you have created your melody with the given instrument in music21, you can export the audio as a `.wav` file, to further edit it (e.g., with Audacity) or to use it in combination with the sound samples (e.g. in EarSketch):

In [ ]:
melody.write('midi', fp = 'frere_jaques_newTimbre.mid')

### Bonus timbre: pure sine waves

On top of using the four timbres (accordion, violin, clarinet and woodblock) directly in music21, you can also play your melodies using pure sine waves. A sine wave is the most elementary type of wave, as all other waves can be obtained by combining sine waves. 

For our purposes, a sine wave is defined simply as a vector of the quantized values of a sine function with given frequency and duration. The quantization is parametrized by the framerate, the number of values retained for every second. The vector is stored as a `numpy` array.

In [ ]:
FRAMERATE = 44100 # <- rate of sampling

In [ ]:
def sine(duration, freq = 440, framerate = FRAMERATE):
    """Returns a sine wave with given frequency (in Hz) and duration (in s), expressed as a vector of quantized values"""
    t = np.linspace(0, duration, int(framerate * duration)) # <- setup time values
    wave = np.sin(2 * np.pi * freq * t)
    
    return wave

For example, here is 1 second of a sine wave with frequency 420 Hz:

In [ ]:
sine(1, 420)

Since the wave is exactly 1s long, the length of the vector is exactly the framerate:

In [ ]:
len(sine(1, 420))

We can play the wave with the following function

In [ ]:
def play_sound(wave, framerate = FRAMERATE):
    #Plays a sound wave expressed as an array
    return Audio(wave, rate = framerate, autoplay=True)


In [ ]:
play_sound(sine(1, 420))

We can easily create sequences of sine waves by concatenating several arrays. We might also want to include some rests (silences) here and there:

In [ ]:
def silence(duration, framerate = FRAMERATE):
    #Returns array of silence
    t = np.linspace(0, duration, int(framerate * duration))
    
    return 0*t

In [ ]:
#Just as an example, a sequence of 1s@420Hz, 2s@220Hz, then 1s of silence, and finally two short 0.5s sounds @330 and 420 Hz.
sequence_of_sounds = np.concatenate([sine(1, 420), sine(2, 220), silence(1), sine(0.5, 330), sine(0.5, 420)])

In [ ]:
play_sound(sequence_of_sounds)

Once we have generated melodies or other pitch materials using the full power of python and music21, we can then transform it into a sequence of sine waves (and silences) with the help of the following function: 

In [ ]:
def notes_to_sines(melody):
    """Transforms a melody (as a music21 stream) into a sequence of sine waves"""
    
    try: #try to extract tempo
        tempo =  melody.duration.quarterLength / melody.seconds #beats per second
    except: #if no tempo is specified, default is 120bpm
        tempo = 2 #beats per second
        
    
    notes = melody.flat.notesAndRests
    
    sines = []
    for x in notes:
        if x.isRest:
            sines += [silence(x.quarterLength / tempo)]
        else:
            sines += [sine(x.quarterLength / tempo, x.pitch.frequency)]
    
    return np.concatenate(sines)

In [ ]:
play_sound(notes_to_sines(melody))

In [ ]:
import os

## Using Earsketch for placing audio samples on a temporal grid

**Given:** a list of up to 10 samples and a trigger map mapping an onset to one of the samples. Every onset stops the playback of a previous sample that might still be playing (several layers can be created that way if you want samples to overlap)

**Goal:** an audio file that contains the samples at the desired positions.

This process can be used to plug together your entire composition, or for producing new samples that can then be used in the next iteration.

### 1. Upload samples to Earsketch

* Head to https://earsketch.gatech.edu/earsketch2/
* In order to upload, you need to create an account.
* On the left side, open the "Sounds" tab and click "Add sound". You should see the upload mask:

<img src="earsketch_upload.png">

* under "Constant Value" add a meaningful name. Note that it will be capitalized and your username will be prepended.
* Scroll down in the sound collection until you see the category with your username. Here you find the names of all your uploaded samples which you can use as variables in the Python code on the right.

### 2. Combine samples and triggermap

<img src="earsketch_playback.png">

* In the code field, the first three lines and the last one always need to be there. We set the tempo to 60 bpm, so that every beat has the duration of 1 second.
* Then we define the list with the `samples` that will be triggered and note the list index of each.
* Now we define a triggermap `onsets`:
  * every position in the string corresponds to 1/4 of a beat, so 0.25 seconds is the lowest resolution (in this tempo)
  * an integer 0-9 causes the sample at this index to be played back
  * every `+` causes the playback to continue, every `-` stands for a rest
* then we combine the `samples` list with the `onsets`, using the function `makeBeat()`. The arguments are:
  * a single sample or list of samples
  * the track (layer) in which the samples are to be played
  * the measure in which to begin the playback
  * the triggerlist
* once defined, we can hit the green "Run" button and see the outcome on top in the Digital Audio Workstation (DAW)
* in the example, you see that
  * the triggermap `onsets` contains 1 for the second sample, `JEYES_GOAT` and 0 for the first sample, `JEYES_HARM`
  * it is put into the first track at measure 2
  * the second call to the `makeBeat` function only receives one sample and therefore only uses 0 in the triggermap, which also contains rests, i.e, for `0-`, only the first 0.25 seconds of the sample are being played.
  
Here are a few lines of code to give you an idea how you can translate your durations expressed in seconds to such a triggermap:

In [ ]:
def duration2earsketch(dur, tempo=60):
    """For a duration in seconds and a tempo, return the number of symbols to put into the trigger map."""
    resolution = 4 * round(tempo) / 60
    return round(float(dur) * resolution)

def earsketch_event(sample_no, positions, sounding_positions=None):
    """Pass the index of the sample to be played, the number you got from duration2earsketch,
    and how many of the positions you want the sample to keep sounding (the rest is filled up with rests)."""
    if sample_no is None:
        return '-' * positions
    positions -= 1
    sounding_positions = positions if sounding_positions is None else sounding_positions - 1
    rest_positions = positions - sounding_positions
    return f"{sample_no}{'+' * sounding_positions}{'-' * rest_positions}"

def durations2triggermap(succession_of_samples, onsets, tempo=60):
    if isinstance(succession_of_samples, int):
        succession_of_samples = [succession_of_samples]
    nxt_sample = (s for s in succession_of_samples)
    result = ''
    for ons in onsets:
        positions = duration2earsketch(ons)
        if isinstance(ons, str):
            result += earsketch_event(None, positions)
        else:
            try:
                sample_no = next(nxt_sample)
            except:
                break
            result += earsketch_event(sample_no, positions)
    return result
            
sample_succession = [0, 1, 0]
durations_in_seconds = [0, '1', 1.75, 6.9]    
durations2triggermap(sample_succession, durations_in_seconds)

### 3. Downloading audio from earsketch

Just go to the "Scripts" tab, find the script where you entered the code and open the menu to select "Download".

<img src="earsketch_download.png" width="300px">